In [1]:
from google.cloud import bigquery

# Kaggle already has Google Cloud credentials set up if you use datasets from BigQuery
client = bigquery.Client(project="gen-lang-client-0538587318")

query = """
SELECT * FROM `gen-lang-client-0538587318.bangkok_traffy.traffy_raw` WHERE timestamp >= '2025-05-05' 
"""
results = client.query(query).to_dataframe()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [2]:
df = results.copy()
df.shape[0]

991

In [3]:
df_todo = df.copy()
df_todo = df_todo[df_todo['photo'] != ""]
df_todo = df_todo[df_todo['ticket_id'] != ""]
df_todo.shape[0]

991

In [4]:
!pip install pandas requests torch transformers scikit-learn torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [5]:
import pandas as pd
import torch
import requests
from PIL import Image
from io import BytesIO
from torchvision import transforms
from transformers import AutoTokenizer, AutoModel, SiglipModel, SiglipVisionModel, AutoImageProcessor, SiglipProcessor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import normalize
from functools import lru_cache

2025-05-06 19:16:46.691873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746559006.892746      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746559006.948438      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

text_model = AutoModel.from_pretrained("BAAI/bge-m3").to(device)
text_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

image_model = (
    SiglipVisionModel.from_pretrained("google/siglip-so400m-patch14-384")
    .eval()
    .to(device)
)
image_processor = SiglipProcessor.from_pretrained("google/siglip-so400m-patch14-384")

_session = requests.Session()
_adapter = requests.adapters.HTTPAdapter(pool_connections=64, pool_maxsize=64)
_session.mount("http://", _adapter)
_session.mount("https://", _adapter)

@lru_cache(maxsize=4096)          # cache repeated URLs (tune size as you like)
def _download_bytes(url: str) -> bytes:
    """Fetch raw bytes with retries & keep‑alive."""
    resp = _session.get(url, timeout=8)
    resp.raise_for_status()
    return resp.content

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

In [7]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np

def get_text_embeddings(texts: list[str]):
    inputs = text_tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        output = text_model(**inputs)
        embeddings = output.last_hidden_state[:, 0]  # CLS token
    return embeddings.cpu().numpy()

def get_image_embeddings(images: list[Image.Image]):
    inputs = image_processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = image_model(**inputs)
    return outputs.pooler_output.cpu().numpy()

def fetch_images(urls: list[str]):
    with ThreadPoolExecutor(max_workers=16) as executor:
        raw_images = list(executor.map(_download_bytes, urls))
    pil_images = [Image.open(BytesIO(raw)).convert("RGB") for raw in raw_images]
    return pil_images

def embed_batch(df_chunk):
    texts = df_chunk["comment"].tolist()
    urls = df_chunk["photo"].tolist()

    text_vecs = get_text_embeddings(texts)
    images = fetch_images(urls)
    image_vecs = get_image_embeddings(images)

    combined_vecs = normalize(np.hstack((text_vecs, image_vecs)))
    return combined_vecs

In [8]:
df_todo

,ticket_id,type,organization,comment,photo,photo_after,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,longitude,latitude
0,2025-HZVY92,ถนน,"กรุงเทพมหานคร, เขตหนองจอก",ขอความอนุเคราะห์ให้ดำเนินการปรับปรุงถนนโดยการร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,461 ซอย อมรทรัพย์ 14 แยก 4 แขวงกระทุ่มราย เขตห...,กระทุ่มราย,หนองจอก,จังหวัดกรุงเทพมหานคร,2025-05-05 17:43:11+00:00,finish,4,0,2025-05-06 13:45:51+00:00,100.86206,13.82676
1,3TLWEW,อื่นๆ,"ร้องทุกข์ กทม. 1555, กรุงเทพมหานคร, เขตบึงกุ่ม...",ปัญหา: ตั้งแต่ปากซอยถึงสุดซอยดังกล่าว พบป้ายโฆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,329 ซอย นวมินทร์ 42 แขวงคลองกุ่ม เขตบึงกุ่ม กร...,คลองกุ่ม,บึงกุ่ม,จังหวัดกรุงเทพมหานคร,2025-05-05 22:52:21+00:00,finish,<NA>,0,2025-05-06 10:37:54+00:00,100.66045,13.79898
2,2025-FCX46U,อุบัติเหตุ,"กรุงเทพมหานคร, เขตบางกะปิ, สำนักการจราจรและขนส...",ทางเข้าออกโครงการไม่มีเส้นกากบาท ทำให้รถเข้าออ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,QJQH+RF4 ซ. ลาดพร้าว 101 แขวงคลองจั่น บางกะปิ ...,คลองจั่น,บางกะปิ,จังหวัดกรุงเทพมหานคร,2025-05-05 22:27:41+00:00,finish,<NA>,0,2025-05-06 13:35:38+00:00,100.62871,13.78954
3,2025-B42UK6,หาบเร่แผงลอย,"กรุงเทพมหานคร, เขตบางกะปิ, ฝ่ายเทศกิจ เขตบางกะปิ",ขายสินค้าบนสะพานลอยหน้ามอลล์ บางกะปิ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,1F - F3 3522 ถ. ลาดพร้าว แขวงคลองจั่น บางกะปิ ...,คลองจั่น,บางกะปิ,จังหวัดกรุงเทพมหานคร,2025-05-05 17:06:43+00:00,finish,<NA>,0,2025-05-06 15:18:40+00:00,100.64266,13.76593
4,2025-3KW3PX,ความสะอาด,"กรุงเทพมหานคร, เขตคลองเตย, ฝ่ายเทศกิจ เขตคลองเตย",ให้เจ้าหน้ามาเก็บขยะริมกำแพงให้หมดด้วยครับ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,61/35 สุขุมวิท แขวงคลองตัน เขตคลองเตย กรุงเทพม...,คลองตัน,คลองเตย,จังหวัดกรุงเทพมหานคร,2025-05-05 09:36:13+00:00,finish,1,0,2025-05-06 08:23:20+00:00,100.56956,13.72528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,2025-MF2BPG,เสียง,"กรุงเทพมหานคร, เขตวัฒนา, ฝ่ายเทศกิจ เขตวัฒนา",5/5/68 16.10 มีรถเร่เปิดท้ายขายแตงโมเสียงดังรบ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,36 ถ. สุขุมวิท 71 แขวงพระโขนง เขตวัฒนา กรุงเทพ...,พระโขนงเหนือ,วัฒนา,จังหวัดกรุงเทพมหานคร,2025-05-05 16:14:13+00:00,start,<NA>,1,2025-05-06 20:22:39+00:00,100.59517,13.71567
987,2025-RF33HN,ผิดกฎจราจร,"กรุงเทพมหานคร, เขตราชเทวี, ฝ่ายเทศกิจ เขตราชเทวี",ร้านขายเสื้อผ้าดังกล่าวมักมีนิสัยกั๊กที่จอดรถข...,https://storage.googleapis.com/traffy_public_b...,,11/59 ถ. ราชปรารภ แขวงมักกะสัน เขตราชเทวี กรุง...,มักกะสัน,ราชเทวี,จังหวัดกรุงเทพมหานคร,2025-05-05 12:36:19+00:00,start,<NA>,0,2025-05-06 10:17:24+00:00,100.54308,13.75362
988,2025-33VCTP,ผิดกฎจราจร,"กรุงเทพมหานคร, เขตราชเทวี, ฝ่ายเทศกิจ เขตราชเทวี",สวัสดี ตามภาพมีพฤติกรรมจองที่จอดรถเป็นระยะเวลา...,https://storage.googleapis.com/traffy_public_b...,,32/4 ถ. ราชปรารภ แขวงมักกะสัน เขตราชเทวี กรุงเ...,มักกะสัน,ราชเทวี,จังหวัดกรุงเทพมหานคร,2025-05-05 12:33:15+00:00,start,<NA>,0,2025-05-06 11:06:42+00:00,100.54324,13.75354
989,2025-2VXPY7,ความสะอาด,"กรุงเทพมหานคร, เขตราชเทวี, ฝ่ายรักษาความสะอาดฯ...",ขยะไม่มาเก็บคับ,https://storage.googleapis.com/traffy_public_b...,,1105/11 ถ. เพชรบุรี แขวงมักกะสัน เขตราชเทวี กร...,มักกะสัน,ราชเทวี,จังหวัดกรุงเทพมหานคร,2025-05-05 06:49:44+00:00,start,<NA>,0,2025-05-06 11:00:36+00:00,100.54527,13.75120


In [9]:
from tqdm import tqdm

batch_size = 128
vectors = []

for i in tqdm(range(0, len(df_todo), batch_size)):
    chunk = df_todo.iloc[i:i+batch_size]
    combined_vecs = embed_batch(chunk)
    vectors.extend(combined_vecs.tolist())

df_todo["vector"] = vectors

100%|██████████| 8/8 [04:41<00:00, 35.23s/it]


In [21]:
project = "gen-lang-client-0538587318"
source_table = "bangkok_traffy.traffy_raw"
destination_table = f"{project}.bangkok_traffy.traffy_vector"

In [22]:
from google.cloud import bigquery

client = bigquery.Client(project=project)

# Append to the table instead of replacing it
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")

job = client.load_table_from_dataframe(df_todo, destination_table, job_config=job_config)
job.result()

print("✅ Upload completed to", destination_table)

✅ Upload completed to gen-lang-client-0538587318.bangkok_traffy.traffy_vector
